# Week 3 Exercise Solution


In [ ]:
# !pip install openai gradio soundfile
import openai
import gradio as gr
import os
import soundfile as sf

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
def transcribe_audio(audio):
    if audio is None:
        return ""
    data, samplerate = sf.read(audio)
    wav_path = "temp.wav"
    sf.write(wav_path, data, samplerate)
    with open(wav_path, "rb") as f:
        transcript = openai.Audio.transcribe("whisper-1", f)
    return transcript['text']

def summarize_transcript(transcript):
    response = openai.ChatCompletion.create(
        model='gpt-4o',
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant that summarizes meeting minutes.'},
            {'role': 'user', 'content': transcript}
        ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
def meeting_minutes_interface(audio):
    transcript = transcribe_audio(audio)
    summary = summarize_transcript(transcript)
    return summary

with gr.Blocks() as demo:
    gr.Markdown("# Meeting Minutes Generator (Week 3)")
    audio = gr.Audio(source="upload", type="filepath", label="Upload Meeting Audio")
    summary = gr.Textbox(label="Meeting Summary", lines=6)
    submit = gr.Button("Generate Summary")
    submit.click(meeting_minutes_interface, inputs=audio, outputs=summary)
demo.launch()